# Practice Model re training

In [3]:
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print(f"TF version: {tf.__version__}")
print(f"Hub version: {hub.__version__}")

with tf.device('/gpu:0'):
    a = tf.Variable([[1., 2.]])
    b = tf.constant([[3.], [4.]])
    print(tf.matmul(a, b))

AlreadyExistsError: Another metric with the same name already exists.

### Model select (already training)

In [8]:
module_selection = ("mobilenet_v2_100_224", 224)
handle_base, pixels = module_selection
MODULE_HANDLE = f"https://tfhub.dev/google/imagenet/{handle_base}/feature_vector/4"
IMAGE_SIZE = (pixels, pixels)
print(f"Using {MODULE_HANDLE} with input size {IMAGE_SIZE}")

BATCH_SIZE = 32


Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


### Set up flower dataset

In [9]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228813984/228813984 [==============================] - 173s 1us/step


### SetUp dataset

In [10]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                       interpolation='bilinear')

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs
)                   
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs
) 

do_data_augmentation = False
if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2, height_shift_range=0.2,
        shear_range=0.2, zoom_range=0.2
        **datagen_kwargs
    )
else:
    train_datagen = valid_datagen
    train_generator = train_datagen.flow_from_directory(
        data_dir, subset="training", shuffle=True, **dataflow_kwargs
    )


Found 731 images belonging to 5 classes.
Found 2939 images belonging to 5 classes.


### Difine model

In [12]:
do_fine_tuning = False

print(f"Building model with {MODULE_HANDLE}")
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3, )),
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.build((None,)+IMAGE_SIZE+(3,))
model.summary()


Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


### training model

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy']
)

c:\Users\hokuto\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In this training program recommends running on the GPU

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

hist = model.fit(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps
).history()

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training steps")
plt.ylim([0, 2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training steps")
plt.ylim([0, 1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

### Check the model using test data

In [ ]:
def get_class_string_from_index(index):
    for class_string, class_index in valid_generator.class_indices.items():
        if class_index == index:
            return class_string

x, y = next(valid_generator)
image = x[0, :, :, :]
true_index = np.argmax(y[0])

### 気になった変数を表示する用
print(x)
print(y)
print(true_index)

plt.imshow(image)
plt.axis('off')
plt.show()

# Expand the validation image to (1, 224, 224, 3) before predicting the label
prediction_scores = model.predict(np.expand_dims(image, axis=0))
predicted_index = np.argmax(prediction_scores)
print(f"True label: {get_class_string_from_index(true_index)}")
print(f"Predicted label: {get_class_string_from_index(predicted_index)}")

### Save the model

In [ ]:
saved_model_path = "./MLmodel"
tf.saved_model.save(model, saved_model_path)